In [1]:
import numpy as np

from orc.branch_bound import *
from orc.utils import generate_problem
from orc.primal import *
from orc.relaxation import *
from orc.reduction import *
from orc.callbacks import *

In [2]:
A, b = generate_problem(10, 20)

In [3]:
bb = BranchAndBound(branch_strategy=branch_strategy2, 
                    lb_strategy=dual_lb,
                    callbacks=[PrimalHeurCallback(), 
                               LagrPenaltiesReductionCallback(),
                               ColumnInclusionCallback()]
                    )

In [4]:
bb.search(A, b)

Set parameter Username
Academic license - for non-commercial use only - expires 2024-06-16
Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (win64)

CPU model: Intel(R) Core(TM) i7-9700K CPU @ 3.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 20 rows, 10 columns and 100 nonzeros
Model fingerprint: 0xd66770a6
Coefficient statistics:
  Matrix range     [9e+00, 2e+02]
  Objective range  [6e+01, 4e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [5e+01, 6e+02]
Presolve removed 1 rows and 0 columns
Presolve time: 0.00s
Presolved: 19 rows, 10 columns, 99 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    4.3167823e+03   2.060869e+02   0.000000e+00      0s
       9    2.0885909e+03   0.000000e+00   0.000000e+00      0s

Solved in 9 iterations and 0.01 seconds (0.00 work units)
Optimal objective  2.088590899e+03
Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (

KeyboardInterrupt: 

Exception ignored in: 'gurobipy.logcallbackstub'
Traceback (most recent call last):
  File "d:\Users\gabri\anaconda3\envs\p3.11\Lib\site-packages\ipykernel\iostream.py", line 480, in flush
    if not evt.wait(self.flush_timeout):
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "d:\Users\gabri\anaconda3\envs\p3.11\Lib\threading.py", line 622, in wait
    signaled = self._cond.wait(timeout)
               ^^^^^^^^^^^^^^^^^^^^^^^^
  File "d:\Users\gabri\anaconda3\envs\p3.11\Lib\threading.py", line 324, in wait
    gotit = waiter.acquire(True, timeout)
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
KeyboardInterrupt: 


Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (win64)

CPU model: Intel(R) Core(TM) i7-9700K CPU @ 3.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 4 rows, 10 columns and 23 nonzeros
Model fingerprint: 0x66796a4d
Coefficient statistics:
  Matrix range     [2e+01, 2e+02]
  Objective range  [1e+01, 8e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [5e+01, 6e+02]
Presolve removed 1 rows and 3 columns
Presolve time: 0.01s
Presolved: 3 rows, 7 columns, 17 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    7.1333333e+02   8.303278e+00   0.000000e+00      0s
       4    6.4252974e+02   0.000000e+00   0.000000e+00      0s

Solved in 4 iterations and 0.01 seconds (0.00 work units)
Optimal objective  6.425297408e+02
Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (win64)

CPU model: Intel(R) Core(TM) i7-9700K CPU @ 3.60GHz, instruction set [SSE2|AVX|AVX2]
Th

In [ ]:
print(bb.best)

In [ ]:
bb.best.x1, str(bb.best)

In [ ]:
x = np.zeros(A.shape[-1])
x[bb.best.x1] = 1
x

In [ ]:
A @ x >= b

In [ ]:
np.sum(A, axis=0) @ x

In [ ]:
bb.node_count

In [ ]:
2 ** 20

In [ ]:
import gurobipy as gp
import numpy as np
from gurobipy import GRB

m = gp.Model("mip")
x = m.addMVar(A.shape[-1], vtype=GRB.BINARY, name="x")
m.setObjective(np.sum(A, axis=0) @ x)
m.addConstr(A @ x >= b)

m.optimize()
res = []
for v in m.getVars():
    res.append(v.x)
m.getObjective().getValue(), res

In [ ]:
x = np.array(res)

In [ ]:
A @ x >= b

In [ ]:
np.sum(A, axis=0) @ x

In [ ]:
a = [1,2,3,4,5]
b = [3,5]
a - b